# The (7,4) Hamming Code

As a preliminary, we define a modulo 2 function.

In [82]:
import numpy as np

For encoding, we need the $\mathbf{G}$ matrix (Eq. 1.28 in MacKay, 2003)

In [83]:

G = np.array([[1, 0, 0, 0, 1, 0, 1],[ 0, 1, 0, 0, 1, 1, 0], [0, 0, 1, 0, 1, 1, 1],[0, 0, 0, 1, 0, 1, 1]])

Now we simply post-multiply $\mathbf{G}$ to the source message $\mathbf{s}$ to get the codeword $\mathbf{t}$. In order to get a binary codeword, we need to take the result modulo 2.

In [84]:
s = np.array([1, 0, 1, 1]) 

In [85]:
#Apply mod2 to all entries
t = s@G
t=t%2
t


array([1, 0, 1, 1, 0, 0, 1], dtype=int32)

The binary symmetric channel flips any transmitted bit with probability $f$. We use this property to generate a noise vector $\mathbf{n}$.

In [86]:
import random
f = 0.1
n = np.random.binomial(1,f,7)

The received vector $\mathbf{r}$ is the sum (modulo 2) of the transmitted codeword $\mathbf{t}$ and the noise $\mathbf{n}$.

In [87]:
r = (t + n)%2
print(t,n,r)

[1 0 1 1 0 0 1] [1 0 0 0 0 0 0] [0 0 1 1 0 0 1]


For decoding, we need the *parity-check matrix* $\mathbf{H}$.

In [88]:
H = np.array([[1, 1, 1, 0, 1, 0, 0],[0, 1, 1, 1, 0, 1, 0], [1, 0, 1, 1, 0, 0, 1]])

This enables us to calculate the *syndrome* $\mathbf{z}$ of the received vector $\mathbf{r}$.

In [89]:
z = r@H.transpose()%2
z

array([1, 0, 1], dtype=int32)

Now we need a function that maps the syndrome $\mathbf{z}$ onto the most likely noise vector $\mathbf{\hat{n}}$ (most likely for small flip rates $f$, that is).

In [90]:
np.all(z==np.array([0, 0, 0])) 

False

In [91]:
def nhat(z):
    if np.all(z==np.array([0, 0, 0])):
        n = np.array([0, 0, 0, 0, 0, 0, 0])
    elif np.all(z==np.array([0, 0, 1])):
        n = np.array([0, 0, 0, 0, 0, 0, 1])
    elif np.all(z==np.array([0, 1, 0])):
        n = np.array([0, 0, 0, 0, 0, 1, 0])
    elif np.all(z==np.array([0, 1, 1])):
        n = np.array([0, 0, 0, 1, 0, 0, 0])
    elif np.all(z==np.array([1, 0, 0])):
        n = np.array([0, 0, 0, 0, 1, 0, 0])
    elif np.all(z==np.array([1, 0, 1])):
        n = np.array([1, 0, 0, 0, 0, 0, 0])
    elif np.all(z==np.array([1, 1, 0])):
        n = np.array([0, 1, 0, 0, 0, 0, 0])
    elif np.all(z==np.array([1, 1, 1])):
        n = np.array([0, 0, 1, 0, 0, 0, 0])
    else:
        print("Input is not a valid syndrome")
    return(n)



In [92]:
nhat(z)

array([1, 0, 0, 0, 0, 0, 0])

Our best guess $\mathbf{\hat{t}}$ for the transmitted vector $\mathbf{t}$ is the sum (modulo 2) of the received vector $\mathbf{r}$ and $\mathbf{\hat{n}}$. If at most one bit was flipped (i.e., $\mathbf{n}$ contained at most one non-zero element), then our guess will be correct. However, if more than one bit was flipped during transmission, our guess will be wrong.

In [93]:
that = (r + nhat(z))%2
print(t)
that

[1 0 1 1 0 0 1]


array([1, 0, 1, 1, 0, 0, 1], dtype=int32)

Finally, our best guess $\mathbf{\hat{s}}$ for the source vector is the first four bits of $\mathbf{\hat{t}}$.

In [98]:
shat = that[0:4]
shat

array([1, 0, 1, 1], dtype=int32)